# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846866074884                   -0.70    4.8
  2   -7.852323292223       -2.26       -1.53    1.0
  3   -7.852646062078       -3.49       -2.52    3.2
  4   -7.852646676389       -6.21       -3.36    2.2
  5   -7.852646685907       -8.02       -4.74    1.5
  6   -7.852646686724       -9.09       -5.21    3.8
  7   -7.852646686729      -11.25       -5.97    1.0
  8   -7.852646686730      -12.44       -7.67    2.0
  9   -7.852646686730      -14.75       -7.48    3.0
 10   -7.852646686730      -14.45       -8.39    2.0
 11   -7.852646686730   +  -14.75       -9.43    2.2
 12   -7.852646686730   +  -14.75       -9.53    1.0
 13   -7.852646686730      -14.45      -10.72    1.8
 14   -7.852646686730   +  -14.57      -11.05    2.5
 15   -7.852646686730      -14.57      -11.68    2.0
 16   -7.852646686730   +    -Inf      -12.04    1.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846854501663                   -0.70           4.8
  2   -7.852526242268       -2.25       -1.64   0.80    2.0
  3   -7.852635854359       -3.96       -2.73   0.80    1.0
  4   -7.852646460839       -4.97       -3.28   0.80    2.2
  5   -7.852646682068       -6.66       -4.13   0.80    1.5
  6   -7.852646686351       -8.37       -4.81   0.80    1.2
  7   -7.852646686722       -9.43       -5.82   0.80    2.0
  8   -7.852646686730      -11.09       -6.74   0.80    2.2
  9   -7.852646686730      -12.98       -7.74   0.80    2.2
 10   -7.852646686730   +  -15.05       -8.71   0.80    2.2
 11   -7.852646686730      -14.45       -8.41   0.80    3.0
 12   -7.852646686730   +  -14.75       -9.46   0.80    1.0
 13   -7.852646686730   +    -Inf       -9.87   0.80    1.2
 14   -7.852646686730   +  -15.05      -10.52   0.80    1.2
 15   -7.852646686730      -14.57      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.401755e+01     3.170813e+00
 * time: 0.4618558883666992
     1     1.011138e+00     1.621464e+00
 * time: 0.7044589519500732
     2    -1.372487e+00     1.952217e+00
 * time: 0.7309598922729492
     3    -3.771051e+00     1.660165e+00
 * time: 0.7698860168457031
     4    -5.024524e+00     1.413719e+00
 * time: 0.8090798854827881
     5    -6.770002e+00     8.144984e-01
 * time: 0.8491299152374268
     6    -6.984151e+00     9.742548e-01
 * time: 0.9693019390106201
     7    -7.561639e+00     6.507033e-01
 * time: 0.9953529834747314
     8    -7.667288e+00     8.144754e-01
 * time: 1.0209259986877441
     9    -7.713306e+00     6.135366e-01
 * time: 1.048063039779663
    10    -7.745640e+00     6.147709e-01
 * time: 1.0743849277496338
    11    -7.778990e+00     4.318149e-01
 * time: 1.1004199981689453
    12    -7.796058e+00     3.214593e-01
 * time: 1.1256890296936035
    13    -7.812623e+00     9.684468e-02
 * time: 1.15140795707

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846845942185                   -0.70    4.8
  2   -7.852320616590       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686709                   -2.56
  2   -7.852646686730      -10.68       -5.96
  3   -7.852646686730   +  -15.05      -12.66


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 7.120171389295844e-13
|ρ_newton - ρ_scfv| = 2.457923387119928e-13
|ρ_newton - ρ_dm|   = 1.1083621904822855e-9
